# The Great Annual Fish Fry Migration, Part 1

![](https://media.giphy.com/media/l0HFjNx0bf4vMgYNi/giphy.gif)

Extract and transform all of last year's fish frys so they're ready for this year. This includes:

* changing Lenten dates to this the corresponding dates for this year
* reseting the validation and publication status

While we could do this all through the Fish Fry API, and update live data, it's trivial to do it all locally and then upload the results. The Fish Fry database is just GeoJSON and is pretty small (<1Mb).

Part 2 of the migration deals with loading the transformed data&mdash;we'll save that for another notebook.

In [1]:
import json
from datetime import datetime, timedelta
from dateutil.parser import parse
from dateutil.easter import easter
from dateutil import tz
import requests

## Helper functions:

In [2]:
def get_fishfry_dates(year):
    """use the dateutil.easter module to get all the Fish Fry dates
    
    Returns a list of iso-formatted dates.
    """
    e = easter(year)
    good_friday = e - timedelta(days=2)
    lenten_fridays = [ good_friday - timedelta(weeks=i + 1) for i in range(0,6)]
    ash_wed = lenten_fridays[-1] - timedelta(days=2)
    dates = []
    return [dt.isoformat() for dt in [good_friday, *lenten_fridays, ash_wed]]

def create_event_migration_lookup(this_year, last_year):
    """creates a dictionary where keys are fish fry dates from last year, 
    and values are for this year
    """
    dates_this_year = get_fishfry_dates(this_year)
    dates_last_year = get_fishfry_dates(last_year)
    return dict(zip(dates_last_year, dates_this_year))

In [3]:
def migrate_date(datetime_str, event_migration_lookup):
    """update a single datetime object using a lookup
    """
    if not datetime_str:
        return None
    
    dt_obj = parse(datetime_str)
    
    key = "{0}-{1}-{2}".format(
        dt_obj.year, str(dt_obj.month).zfill(2),str(dt_obj.day).zfill(2)
    )
    
    if key in event_migration_lookup.keys():
        
        result = "{0}T{1}:{2}:00".format(
            # construct ISO8061-formatted string with new date from lookup, 
            # but with previously recorded time of day (minus the timezone)
            event_migration_lookup[key], dt_obj.hour, dt_obj.minute
        )
        dt_obj2 = parse(result)
        # assign the timezone. This ensures we account for DST!
        dt_obj2 = dt_obj2.replace(tzinfo=tz.gettz('America/New_York'))
        
        return dt_obj2.isoformat()
    
    # return the original datetime object if it wasn't found in the lookup
    return dt_obj.isoformat()

In [4]:
def run_migration_part1(
    fishfry_geojson, 
    event_migration_lookup
):
    """takes a complete fishfry geojson and an event lookup, and returns an updated geojson
    """

    updated_features = []

    for f in fishfry_geojson['features']:

        # create a new geojson feature, with the existing ID and geometry
        feature = {
            'id': f['id'],
            'geometry': f['geometry'],
            'properties' : {}
        }
        
        # copy over all properties except for events to the new feature
        for k, v in f['properties'].items():
            if k not in ['events']:
                if v != "":
                    feature['properties'][k] = v
                else:
                    # Note that missing values can't be empty strings.
                    # This helps when loading back to DynamoDB.
                    feature['properties'][k] = None

        # reset validation/publication properties
        feature['properties']['validated'] = False
        feature['properties']['publish'] = False

        # transform the events property
        old_events = []
        updated_events = []
        
        for e in f['properties']['events']:
            
            # get the existing datetime ISO strings
            old_d0 = e["dt_start"]
            old_d1 = e["dt_end"]
            
            # transform old datetimes to new dates w/ previous times
            updated_d0 = migrate_date(old_d0, event_migration_lookup)
            updated_d1 = migrate_date(old_d1, event_migration_lookup)
            
            # append it to the updated events list
            updated_events.append({"dt_start": updated_d0, "dt_end": updated_d1})

        # add the events to the feature
        feature['properties']["events"] = updated_events
        
        # push it to the feature list
        feature['type'] = "Feature"
        updated_features.append(feature)

    updated_geojson = {
        "type": "FeatureCollection",
        "features": updated_features
    }

    return updated_geojson

## Ready, set, go

![](https://media.giphy.com/media/lPuW5AlR9AeWzSsIqi/giphy.gif)

### Get a copy of the the dataset from the FishFry API

In [5]:
r = requests.get("https://fishfry.codeforpgh.com/api/fishfries/")
fishfry_geojson = r.json()

### Generate the date lookup

Ash Wednesday, all Lenten Fridays, and Good Friday: last year vs. this year:

In [6]:
event_migration_lookup = create_event_migration_lookup(
    this_year = datetime.now().year,
    last_year = datetime.now().year - 1
)
event_migration_lookup

{'2019-04-19': '2020-04-10',
 '2019-04-12': '2020-04-03',
 '2019-04-05': '2020-03-27',
 '2019-03-29': '2020-03-20',
 '2019-03-22': '2020-03-13',
 '2019-03-15': '2020-03-06',
 '2019-03-08': '2020-02-28',
 '2019-03-06': '2020-02-26'}

### Update it!

In [7]:
updated_fishfrys = run_migration_part1(
    fishfry_geojson, 
    event_migration_lookup
)

Save the results to disk:

In [8]:
with open('updated_fishfrydb.geojson', 'w') as fp:
    json.dump(updated_fishfrys, fp)